In [5]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
import tqdm
import random
import pandas as pd

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import HuberRegressor
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import PassiveAggressiveRegressor

In [7]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

In [8]:
from preprocess import preprocess

X_train, X_test, y_train = preprocess(df_train, df_test, use_ohe=True, use_scaling=True)

In [ ]:
objects_num = 10000

X = X_train.values[:objects_num]
y = y_train[:objects_num]

### Hyperopt

In [12]:
from hyperopt.pyll import scope as ho_scope
from hyperopt import fmin, tpe, hp

### Huber regressor

In [10]:
iter_num = 100

In [11]:
def f(space):
    model = HuberRegressor(epsilon=space['epsilon'], max_iter=space['max_iter'], alpha=space['alpha'])
    rskf = StratifiedKFold(n_splits=5, random_state=1)
    rmse_scores = []
    for train_index, test_index in rskf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, np.round(model.predict(X_test)))))
    return np.mean(rmse_scores)
    
space = {
    'epsilon':  hp.loguniform('epsilon', low=np.log(1.1), high=np.log(10)),
    'max_iter': ho_scope.int(hp.quniform('max_iter', low=100, high=500, q=10)),
    'alpha':  hp.loguniform('alpha', low=np.log(0.0001), high=np.log(0.01)),
}

best = fmin(
    fn=f,  # "Loss" function to minimize
    space=space,  # Hyperparameter space
    algo=tpe.suggest,  # Tree-structured Parzen Estimator (TPE)
    max_evals=100  # Perform 100 trials
)

print("Found minimum after 100 trials:")
print(best)

100%|██████████| 100/100 [09:05<00:00,  5.01s/it, best loss: 1.385947980087629]
Found minimum after 100 trials:
{'alpha': 0.0050020129731213475, 'epsilon': 8.675445556078893, 'max_iter': 290.0}


### Ridge regressor

In [12]:
def f(space):
    model = Ridge(solver=space['solver'], max_iter=space['max_iter'], alpha=space['alpha'])
    rskf = StratifiedKFold(n_splits=5, random_state=1)
    rmse_scores = []
    for train_index, test_index in rskf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, np.round(model.predict(X_test)))))
    return np.mean(rmse_scores)
    
space = {
    'solver': hp.choice('solver', ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']),
    'max_iter': ho_scope.int(hp.quniform('max_iter', low=1000, high=5000, q=100)),
    'alpha':  hp.loguniform('alpha', low=np.log(0.0001), high=np.log(1)),
}

best = fmin(
    fn=f,  # "Loss" function to minimize
    space=space,  # Hyperparameter space
    algo=tpe.suggest,  # Tree-structured Parzen Estimator (TPE)
    max_evals=100  # Perform 100 trials
)

print("Found minimum after 100 trials:")
print(best)

100%|██████████| 100/100 [17:54<00:00,  1.12it/s, best loss: 1.17311775384483]
Found minimum after 100 trials:
{'alpha': 0.6113206240339772, 'max_iter': 3900.0, 'solver': 0}


### Lasso regressor

In [13]:
def f(space):
    model = Lasso(max_iter=space['max_iter'], alpha=space['alpha'], normalize=space['normalize'])
    rskf = StratifiedKFold(n_splits=5, random_state=1)
    rmse_scores = []
    for train_index, test_index in rskf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, np.round(model.predict(X_test)))))
    return np.mean(rmse_scores)
    
space = {
    'normalize': hp.choice('normalize', [True, False]),
    'max_iter': ho_scope.int(hp.quniform('max_iter', low=1000, high=5000, q=100)),
    'alpha':  hp.loguniform('alpha', low=np.log(0.0001), high=np.log(1)),
}

best = fmin(
    fn=f,  # "Loss" function to minimize
    space=space,  # Hyperparameter space
    algo=tpe.suggest,  # Tree-structured Parzen Estimator (TPE)
    max_evals=100  # Perform 100 trials
)

print("Found minimum after 100 trials:")
print(best)

  5%|▌         | 5/100 [00:04<01:27,  1.08it/s, best loss: 1.182254623254029]

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 51246.2233594651, tolerance: 14.081483657091429
  positive)

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 50352.361803329004, tolerance: 14.082399218749995
  positive)

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 51147.74832348185, tolerance: 14.082399218749998
  positive)

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You migh

  6%|▌         | 6/100 [00:23<10:09,  6.48s/it, best loss: 1.172803344462741]

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 51346.39285991386, tolerance: 14.082600767490405
  positive)



 10%|█         | 10/100 [00:26<03:07,  2.08s/it, best loss: 1.172803344462741]

...

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20265.804367593242, tolerance: 14.082600767490405
  positive)



 99%|█████████▉| 99/100 [18:03<00:06,  6.73s/it, best loss: 1.172692796316261]

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 50744.00093964551, tolerance: 14.081483657091429
  positive)

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 49331.28342759957, tolerance: 14.082399218749995
  positive)

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 50244.57854887148, tolerance: 14.082399218749998
  positive)

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You migh

100%|██████████| 100/100 [18:19<00:00,  9.43s/it, best loss: 1.172692796316261]
Found minimum after 100 trials:
{'alpha': 0.0004083806282524149, 'max_iter': 4000.0, 'normalize': 1}


/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 50827.271179211886, tolerance: 14.082600767490405
  positive)



### Decision tree

In [15]:
def f(space):
    model = DecisionTreeRegressor(max_depth=space['max_depth'], min_samples_split=space['min_samples_split'],
                                       min_samples_leaf=space['min_samples_leaf'], min_weight_fraction_leaf=
                                        space['min_weight_fraction_leaf'], max_features=space['max_features'])
    rskf = StratifiedKFold(n_splits=5, random_state=1)
    rmse_scores = []
    for train_index, test_index in rskf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, np.round(model.predict(X_test)))))
    return np.mean(rmse_scores)
    
space = {
    'max_depth':  ho_scope.int(hp.quniform('max_iter', low=4, high=100, q=2)),
    'min_samples_split': ho_scope.int(hp.quniform('min_samples_split', low=2, high=10, q=1)),
    'min_samples_leaf':  ho_scope.int(hp.quniform('min_samples_leaf', low=1, high=10, q=1)),
    'min_weight_fraction_leaf': hp.uniform('min_weight_fraction_leaf', 0, 0.5),
    'max_features': hp.choice('max_features', ['auto', 'sqrt', 'log2'])
}

best = fmin(
    fn=f,  # "Loss" function to minimize
    space=space,  # Hyperparameter space
    algo=tpe.suggest,  # Tree-structured Parzen Estimator (TPE)
    max_evals=100  # Perform 100 trials
)

print("Found minimum after 100 trials:")
print(best)

100%|██████████| 100/100 [03:51<00:00,  2.32s/it, best loss: 1.0292585185873997]
Found minimum after 100 trials:
{'max_features': 0, 'max_iter': 36.0, 'min_samples_leaf': 3.0, 'min_samples_split': 2.0, 'min_weight_fraction_leaf': 0.001560875278310783}


### Passive aggressive

In [16]:
def f(space):
    model = PassiveAggressiveRegressor(max_iter=space['max_iter'], tol=space['max_iter'], 
                                       C = space['C'])
    rskf = StratifiedKFold(n_splits=5, random_state=1)
    rmse_scores = []
    for train_index, test_index in rskf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, np.round(model.predict(X_test)))))
    return np.mean(rmse_scores)
    
space = {
    'max_iter': ho_scope.int(hp.quniform('max_iter', low=1000, high=5000, q=100)),
    'tol': hp.loguniform('tol', low=np.log(0.000001), high=np.log(0.001)),
    'verbose': ho_scope.int(hp.quniform('verbose', low=1, high=100, q=2)),
    'C':  hp.loguniform('C', low=np.log(0.0001), high=np.log(10)),
}

best = fmin(
    fn=f,  # "Loss" function to minimize
    space=space,  # Hyperparameter space
    algo=tpe.suggest,  # Tree-structured Parzen Estimator (TPE)
    max_evals=100  # Perform 100 trials
)

print("Found minimum after 100 trials:")
print(best)

100%|██████████| 100/100 [02:02<00:00,  1.28s/it, best loss: 1.624647185516967]
Found minimum after 100 trials:
{'C': 1.3636978449393053, 'max_iter': 3600.0, 'tol': 3.5018039256719656e-05, 'verbose': 64.0}


### Xgboost

In [14]:
objects_num = 4000

X = X_train[:objects_num]
y = y_train[:objects_num]

In [15]:
def f(space):
    model = xgb.XGBRegressor(objective="reg:linear", booster=space['booster'], eta=space['eta'], 
                               gamma=space['gamma'], max_depth=space['max_depth'], reg_lambda=space['lambda'],
                               alpha=space['alpha'], verbosity=0)
    rskf = StratifiedKFold(n_splits=5, random_state=1)
    rmse_scores = []
    for train_index, test_index in rskf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, np.round(model.predict(X_test)))))
    return np.mean(rmse_scores)
    
space = {
    'booster': hp.choice('booster', ['gbtree', 'gblinear', 'dart']),
    'eta': hp.loguniform('eta', low=np.log(0.001), high=np.log(1)),
    'gamma': hp.loguniform('gamma', low=np.log(0.001), high=np.log(100)),
    'max_depth': ho_scope.int(hp.quniform('max_depth', low=5, high=50, q=2)),
    'lambda': hp.loguniform('lambda', low=np.log(0.001), high=np.log(10)),
    'alpha':  hp.loguniform('alpha', low=np.log(0.001), high=np.log(10)),
}

best = fmin(
    fn=f,  # "Loss" function to minimize
    space=space,  # Hyperparameter space
    algo=tpe.suggest,  # Tree-structured Parzen Estimator (TPE)
    max_evals=100  # Perform 100 trials
)

print("Found minimum after 100 trials:")
print(best)

  0%|          | 0/100 [00:00<?, ?it/s, best loss: ?]

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  

  1%|          | 1/100 [00:01<02:09,  1.31s/it, best loss: 1.2018603253169553]

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  

  2%|▏         | 2/100 [00:02<02:06,  1.29s/it, best loss: 1.160983962157688] 

...

 98%|█████████▊| 98/100 [28:19<00:23, 11.60s/it, best loss: 1.0344876283533544]

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  

 99%|█████████▉| 99/100 [28:42<00:15, 15.16s/it, best loss: 1.0344876283533544]

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  

100%|██████████| 100/100 [29:09<00:00, 18.62s/it, best loss: 1.0344876283533544]
Found minimum after 100 trials:
{'alpha': 0.03166249146358193, 'booster': 0, 'eta': 0.015086009654781104, 'gamma': 11.424190368034198, 'lambda': 0.5211046123217, 'max_depth': 14.0}


In [ ]:
### 